In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [3]:
with ZipFile('data2.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('final.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [4]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [5]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,4909294510,,,нет . . . у тебя на лице написано - нет .,0,тогда я попытался с двумя другими женщинами и ...
1,4909294510,,,нет . . . у тебя на лице написано - нет .,1,"я улыбаюсь потому . . . потому что , описывая ..."
2,4909294510,,,нет . . . у тебя на лице написано - нет .,2,"это , так сказать , соответствует уровню моей ..."
3,4909294510,,,нет . . . у тебя на лице написано - нет .,3,я врач .
4,4909294510,,,нет . . . у тебя на лице написано - нет .,4,не обращайте на меня внимания !


In [6]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [7]:
pd_public.count()

context_id    104834
context_2     104834
context_1     104834
context_0     104834
reply_id      104834
reply         104834
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [10]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (104834, 6), (202367, 8))

In [11]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.518039
 0.0    0.171817
 1.0    0.055612
 2.0    0.254533
Name: context_id, dtype: float64

# Простой бейзлайн

In [12]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
import re

In [14]:
w2v_fpath = "w2v/tenth.norm-sz500-w7-cb0-it5-min5.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

In [15]:
for word, score in w2v.most_similar(u"звони"):
    print (word, score)

позвони 0.844639778137207
позвонишь 0.7767298221588135
позвоню 0.7671072483062744
звонить 0.7096108794212341
звоните 0.7036464810371399
— звони 0.6965322494506836
перезвони 0.6919037103652954
– звони 0.675399124622345
приезжай 0.674954891204834
позванивай 0.6720143556594849


In [16]:
len(w2v.vocab)

2641862

In [17]:
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]] \
    .apply(lambda x: " ".join(x), axis=1)

In [18]:
def clean_text(x):
    x = x.lower()
    x = x.replace("ё", "е")
    return x

In [19]:
empty_vector = np.zeros(w2v.vector_size)
def condext_to_vec(x):
    words = re.findall(r"\b(\w+)\b", clean_text(x))
    vectors = sum([w2v[word] for word in words if word in w2v])
    return (empty_vector + vectors).reshape(1, -1)

In [20]:
def not_in_vec(x):
    words = re.findall(r"\b([a-zа-я]+)\b", clean_text(x))
    words = [word for word in words if word not in w2v]
    return " ".join(words)

In [21]:
s_strange = s_words.apply(not_in_vec)

In [22]:
len(s_strange[~s_strange.eq("")])

2146

In [23]:
s_words.loc[2600]

'ќе шучу . " ты разрешишь мне вернутьс€ в тюрьму ? я начинаю уставать , это так унизительно . когда я возбуждаюсь , я издаю такого рода звуки . раньше ты не лаял .'

In [24]:
s_strange[~s_strange.eq("")]

193          вудхалл
624           фабрей
867            ворус
868            ворус
869            ворус
870            ворус
871            ворус
872            ворус
1109          волмов
1110          волмов
1111          волмов
1112          волмов
1113          волмов
1114          волмов
1952        эрствайл
2166           дэнно
2346          гендзи
2549        оливерио
3025         огустин
3026         огустин
3027         огустин
3028         огустин
3029         огустин
3030         огустин
3232          изобэл
3301          тоетие
3302          тоетие
3303          тоетие
3304          тоетие
3305          тоетие
             ...    
200210      андреску
200211      андреску
200212      андреску
200297         бичум
200298         бичум
200299         бичум
200300         бичум
200301         бичум
200302         бичум
200938       далеков
200959          mама
200960          mама
200961          mама
200962          mама
200963          mама
200964          mама
201283      с

# Сокращаем набор векторов

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
cv = CountVectorizer(token_pattern=r"\b([[a-zа-я]+|\s])\b")

In [27]:
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]].apply(lambda x: clean_text(" ".join(x)), axis=1)

In [28]:
cv.fit(s_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\b([[a-zа-я]+|\\s])\\b',
        tokenizer=None, vocabulary=None)

In [29]:
w2vs = dict(((word, w2v[word]) for word in cv.vocabulary_ if word in w2v))

In [30]:
import pickle

In [31]:
pickle.dump(w2vs, open("w2v/w2vs.pickle", "wb"))

In [32]:
w2vs["привет"]

array([ 6.37970492e-03, -1.34650664e-02,  1.99430902e-03, -1.95421930e-02,
        3.65548767e-02, -1.36225261e-02, -3.68371084e-02,  3.96759622e-02,
       -7.56043047e-02,  6.19350187e-02, -2.03065742e-02,  1.36683707e-03,
        1.10376343e-01, -3.52849737e-02, -2.16800608e-02, -6.76287264e-02,
       -4.43975702e-02, -5.50352670e-02,  3.85433026e-02, -3.27817188e-03,
        1.24752410e-02, -2.73274966e-02,  6.06485084e-02, -3.20205949e-02,
        8.68981481e-02,  5.24307191e-02,  6.98179938e-03, -1.43205347e-02,
        2.09773593e-02, -2.23325659e-02, -8.83859023e-03, -5.72893173e-02,
        4.98229600e-02, -3.86953503e-02,  2.99547222e-02, -1.35326870e-02,
       -4.13755737e-02,  6.57960996e-02, -2.18175817e-03,  6.78563863e-03,
       -9.45874453e-02, -8.67071003e-02, -5.35133742e-02, -6.05945699e-02,
        2.40915082e-02, -3.01319137e-02, -3.93458903e-02,  1.35511886e-02,
       -8.22223574e-02, -1.93500277e-02,  5.16408905e-02,  2.68021878e-02,
        5.70873693e-02,  